<a href="https://colab.research.google.com/github/sangbin0106/CP2/blob/main/CP2_Rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import os
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def norm(tab): 
    if len(tab) > 1:
        max  = np.array(tab).max()
        min = np.array(tab).min()
        return [round((x - min) / (max-min) ,3) for x in tab]
    else : 
        return [1]

def norm_categ_lvl2(dict_categ): 
    for k,v in dict_categ.items(): 
        tab = norm([v for k,v  in v.items()])
        dict_categ[k] ={k:v for (k,_),v in zip(v.items(),tab)}
    return dict_categ

In [4]:
df = pd.read_csv('/content/drive/My Drive/cp2/2019-Oct.csv')

In [5]:
df.dropna(inplace=True)
df["category_lvl1"] = df["category_code"].apply(lambda x : x.split(".")[0])
df["category_lvl2"] = df["category_code"].apply(lambda x : x.split(".")[1])
df.reset_index(drop = True,inplace=True)

only_purchases = df.loc[df.event_type == 'purchase']

In [6]:
purchases_with_brands = only_purchases.loc[only_purchases.brand.notnull()]
top_sellers = purchases_with_brands.groupby('brand')["price", "brand"].sum().sort_values(by="price", ascending=False)
top_sellers.reset_index(inplace=True)
top_sellers.rename(columns={"price":"gain/brand"}, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [7]:
purchases_with_brands = only_purchases.loc[only_purchases.brand.notnull()]
top_sellers_count = purchases_with_brands.groupby('brand').brand.agg([len]).sort_values(by="len", ascending=False)
top_sellers_count.reset_index(inplace=True)
top_sellers_count.rename(columns={"len" : "sales/brand"}, inplace=True)

In [8]:
top_sellers_count

,brand,sales/brand
0,samsung,171706
1,apple,142577
2,xiaomi,46595
3,huawei,23294
4,oppo,10891
...,...,...
1047,gretsch,1
1048,greentree,1
1049,schneider,1
1050,globex,1


In [9]:
sales_list  = [x[1] for x in zip(top_sellers_count["brand"], top_sellers_count["sales/brand"])]
n_sales_brand_norm =pd.Series( norm(sales_list),name="sales/brand")
n_sales_brand_norm = pd.concat([top_sellers_count["brand"], n_sales_brand_norm],axis=1)
n_sales_brand_norm

,brand,sales/brand
0,samsung,1.000
1,apple,0.830
2,xiaomi,0.271
3,huawei,0.136
4,oppo,0.063
...,...,...
1047,gretsch,0.000
1048,greentree,0.000
1049,schneider,0.000
1050,globex,0.000


In [10]:
category_lvl1_purshace = purchases_with_brands.groupby("category_lvl1")["category_lvl1"].agg([len]).sort_values(by="len", ascending=False)
category_lvl1_purshace.reset_index(inplace=True)
category_lvl1_purshace.rename(columns={"len":"sales/category"}, inplace=True)

In [11]:
n_sales_categ_lvl1  = [x[1] for x in zip(category_lvl1_purshace["category_lvl1"], category_lvl1_purshace["sales/category"])]
n_sales_categ_lvl1_norm = norm(n_sales_categ_lvl1)
categ_lvl1_norm = pd.concat([category_lvl1_purshace["category_lvl1"],pd.Series(n_sales_categ_lvl1_norm,name="weight_lvl1")],axis=1)
categ_lvl1_norm

,category_lvl1,weight_lvl1
0,electronics,1.000
1,appliances,0.169
2,computers,0.064
3,auto,0.019
4,apparel,0.014
5,construction,0.013
6,kids,0.011
7,furniture,0.009
8,accessories,0.003
9,sport,0.002


In [12]:
category_lvl2_ = purchases_with_brands.groupby(["category_lvl1","category_lvl2"])["category_lvl2"].agg([len])
category_lvl2_.reset_index(inplace=True)

In [13]:
categories =set(category_lvl2_["category_lvl1"])
dict_categ = {}
for categ in categories: 
    dict_categ[categ] = dict(np.array(category_lvl2_[category_lvl2_["category_lvl1"]== categ]  [  ["category_lvl2",'len']  ]))
dict_categ_norm = norm_categ_lvl2(dict_categ)

categ_lvl2_norm = []
for k,v in dict_categ_norm.items():
    [categ_lvl2_norm.append((k_,v_)) for k_,v_ in v.items()]

categ_lvl2_norm = dict(categ_lvl2_norm)
categ_lvl2_norm

{'accessories': 1,
 'audio': 0.101,
 'bag': 1.0,
 'bathroom': 0.133,
 'bedroom': 1.0,
 'belt': 0.001,
 'bicycle': 1.0,
 'camera': 0.0,
 'carriage': 1.0,
 'cartrige': 1,
 'clocks': 0.048,
 'components': 0.0,
 'costume': 0.013,
 'cultivator': 0.0,
 'desktop': 0.183,
 'dolls': 0.0,
 'dress': 0.001,
 'ebooks': 0.0,
 'environment': 0.378,
 'fmcg': 0.3,
 'iron': 0.067,
 'ironing_board': 0.0,
 'jacket': 0.0,
 'jeans': 0.0,
 'jumper': 0.0,
 'kitchen': 0.833,
 'lawn_mower': 1.0,
 'living_room': 0.831,
 'notebook': 1.0,
 'peripherals': 0.308,
 'personal': 0.041,
 'scarf': 0.002,
 'sewing_machine': 0.029,
 'shirt': 0.006,
 'shoes': 1.0,
 'shorts': 0.0,
 'skates': 0.224,
 'ski': 0.013,
 'skirt': 0.0,
 'smartphone': 1.0,
 'snowboard': 0.0,
 'sock': 0.002,
 'swing': 0.022,
 'tablet': 0.015,
 'telephone': 0.01,
 'tennis': 0.0,
 'tools': 1,
 'toys': 0.325,
 'trainer': 0.261,
 'trousers': 0.001,
 'tshirt': 0.006,
 'umbrella': 0.0,
 'underwear': 0.016,
 'universal': 0.0,
 'video': 0.063,
 'wallet': 0.26

In [14]:
keys = pd.Series([k for k,_ in categ_lvl2_norm.items()],name="category_lvl2")
values = pd.Series([v for  _,v in categ_lvl2_norm.items()],name="weight_lvl2")
categ_lvl2_norm = pd.concat([keys,values],axis=1)
categ_lvl2_norm

,category_lvl2,weight_lvl2
0,audio,0.101
1,camera,0.000
2,clocks,0.048
3,smartphone,1.000
4,tablet,0.015
5,telephone,0.010
6,video,0.063
7,components,0.000
8,desktop,0.183
9,ebooks,0.000


In [15]:
product_sales = purchases_with_brands.groupby("product_id")["product_id"].agg([len]).sort_values(by="len", ascending=False)
product_sales.reset_index(inplace=True)
product_sales.rename(columns={"len":"sales/product"}, inplace=True)
product_sales

,product_id,sales/product
0,1004856,28944
1,1004767,21806
2,1004833,12697
3,1005115,12543
4,4804056,12381
...,...,...
16732,12400467,1
16733,12400462,1
16734,3700952,1
16735,12400437,1


In [16]:
def transform_y_to_classes(x):
    if (x < 10):
        return "1"  #낮음
    elif (x < 40):
        return "2"
    elif (x < 80):
        return "3"
    elif (x < 200):
        return "4"
    elif (x < 1000):
        return "5"
    else:
        return "6" #좋음

In [17]:
product_sales["product rate"] = product_sales["sales/product"].apply(lambda x : transform_y_to_classes(x))
product_sales

,product_id,sales/product,product rate
0,1004856,28944,6
1,1004767,21806,6
2,1004833,12697,6
3,1005115,12543,6
4,4804056,12381,6
...,...,...,...
16732,12400467,1,1
16733,12400462,1,1
16734,3700952,1,1
16735,12400437,1,1


In [18]:
df = pd.merge(df,n_sales_brand_norm,on="brand")
df = pd.merge(df, categ_lvl1_norm,on="category_lvl1")
df = pd.merge(df,categ_lvl2_norm,on="category_lvl2")
df = pd.merge(df,product_sales.drop("sales/product", axis=1),on="product_id")
df.sample(8)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,category_lvl1,category_lvl2,sales/brand,weight_lvl1,weight_lvl2,product rate
5278314,2019-10-24 10:51:10 UTC,view,1307519,2053013558920217191,computers.notebook,acer,1209.55,563387743,847e84ea-bbd5-4d1d-92f8-dd3852de29b1,computers,notebook,0.040,0.064,1.000,1
12202450,2019-10-23 17:48:52 UTC,view,1003768,2053013555631882655,electronics.smartphone,huawei,485.54,530016055,c53c7d44-d0be-426e-958a-a9b7d94237c6,electronics,smartphone,0.136,1.000,1.000,4
11074404,2019-10-15 14:14:35 UTC,purchase,1002547,2053013555631882655,electronics.smartphone,apple,463.59,514580198,320f80b5-8889-4907-ad12-240e59558467,electronics,smartphone,0.830,1.000,1.000,6
61319,2019-10-02 11:55:53 UTC,view,3701277,2053013565983425517,appliances.environment.vacuum,elenberg,41.16,533070301,4644359d-22b4-438a-a6a3-12df743fe4a2,appliances,environment,0.032,0.169,0.378,4
14017826,2019-10-14 03:15:05 UTC,cart,1004836,2053013555631882655,electronics.smartphone,samsung,229.99,559664237,030d5035-ab59-4c44-b165-016fab2eaec3,electronics,smartphone,1.000,1.000,1.000,6
10948906,2019-10-21 09:32:29 UTC,view,1002540,2053013555631882655,electronics.smartphone,apple,462.05,561096141,19a7776a-af02-4149-9898-20052caa9b91,electronics,smartphone,0.830,1.000,1.000,5
22867045,2019-10-22 21:24:56 UTC,view,12400052,2053013556252639687,construction.tools.drill,bosch,102.71,559134706,fe7cad61-bafa-4b04-9420-22f21fef14f1,construction,tools,0.024,0.013,1.000,2
10371855,2019-10-08 20:51:18 UTC,view,1004227,2053013555631882655,electronics.smartphone,apple,1005.35,519079400,07d5070a-c514-497f-8903-2fea14f8da5c,electronics,smartphone,0.830,1.000,1.000,6


In [19]:
Final_DF= df[df["event_type"]=="purchase"]
col_to_drop = ["category_id", "product_id", "category_code","user_id","user_session", "event_type","category_lvl1","category_lvl2","brand","event_time"]
Final_DF = Final_DF.drop(col_to_drop,axis=1)
Final_DF.reset_index(inplace=True, drop=True)
Final_DF.sample(8)

,price,sales/brand,weight_lvl1,weight_lvl2,product rate
362643,380.83,1.000,1.000,1.000,5
86040,849.42,0.011,0.064,1.000,2
386303,245.21,1.000,1.000,1.000,6
107577,192.26,0.830,1.000,0.101,6
521971,51.22,0.004,0.014,1.000,1
436702,228.52,0.271,1.000,1.000,6
141018,139.00,0.000,1.000,0.101,1
527141,142.84,0.000,0.014,1.000,1


In [20]:
Final_DF["product_weight"] = Final_DF["weight_lvl1"] * Final_DF["weight_lvl2"]
Final_DF.sample(20)

,price,sales/brand,weight_lvl1,weight_lvl2,product rate,product_weight
198779,643.23,0.830,1.000,1.000,6,1.000000
94281,272.54,0.008,0.064,0.308,5,0.019712
151896,462.56,0.830,1.000,1.000,6,1.000000
154287,460.55,0.830,1.000,1.000,6,1.000000
1084,25.71,0.032,0.169,0.378,4,0.063882
52121,261.05,0.022,0.169,0.833,5,0.140777
307337,370.06,1.000,1.000,1.000,6,1.000000
303152,388.43,1.000,1.000,1.000,6,1.000000
76574,2496.59,0.830,0.064,1.000,3,0.064000
115800,160.22,0.830,1.000,0.101,6,0.101000
